<center><h1> This is Section 1 for the Segmenting and Clustering Neighbourhoods in Toronto Assignment</h1>

<hr>

Install tool packages:

In [1]:
#!conda install -c conda-forge geopy --yes
#!conda install -c conda-forge folium=0.5.0 --yes

Installing new update for conda package:

In [2]:
#!conda update -n base -c defaults conda

Importing Libraries and Packages:

In [3]:
#!pip install beautifulsoup4 
#!pip install lxml 

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

from bs4 import BeautifulSoup #  library for pulling data out of HTML and XML files

import lxml # parsing websites into lxml format
import requests # library to handle requests
import csv # allows python to parse csv files
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Imported')

Imported


Obtaining the souce code for the website in text format:

In [4]:
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
#source

Running the website text through the BeautifulSoup library to create an object:

In [5]:
soup=BeautifulSoup(source,'lxml')
#soup

Isolating the postal code table:

In [6]:
table=soup.find('table', class_='wikitable sortable')
#table

Defining the table headers:

In [7]:
def get_table_headers(table):
    headers=[]
    for th in table.find('tr').find_all('th'):
        headers.append(th.text.strip())
    return headers

head=get_table_headers(table)
print(head)

['Postal Code', 'Borough', 'Neighbourhood']


Defining the table rows:

In [8]:
def get_table_rows(table):
    rows=[]
    for tr in table.find_all('tr')[1:]:
        cells=[]
        tdr=tr.find_all('td')
        if len(tdr)==0:
            thr=tr.find_all('th')
            for th in thr:
                cells.append(td.text.strip())
        else:
            for td in tdr:
                cells.append(td.text.strip())
        rows.append(cells)
    return rows

table_rows=get_table_rows(table)
#table_rows

Creating a csv file:

In [9]:
def save_as_csv(table_name, headers, rows):
    pd.DataFrame(rows, columns=headers).to_csv(f"{table_name}.csv")

save_as_csv("gta_table", head, table_rows)

Loading the created gta_table.csv data:

In [10]:
df = pd.read_csv('gta_table.csv')
df.head()

,Unnamed: 0,Postal Code,Borough,Neighbourhood
0,0,M1A,Not assigned,Not assigned
1,1,M2A,Not assigned,Not assigned
2,2,M3A,North York,Parkwoods
3,3,M4A,North York,Victoria Village
4,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Removing the rows with 'Not assigned':

In [11]:
cdf=df[df.Borough !='Not assigned']
cdf.head()

,Unnamed: 0,Postal Code,Borough,Neighbourhood
2,2,M3A,North York,Parkwoods
3,3,M4A,North York,Victoria Village
4,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Dropping the 'Unnamed: 0' column:

In [12]:
cdf=cdf.drop('Unnamed: 0', axis=1)
cdf.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Table grouped by Postal Code:

In [13]:
final_df=cdf.groupby(['Postal Code'], sort=False).agg(','.join)
final_df.reset_index(inplace=True)
final_df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [14]:
final_df['Neighbourhood']=np.where(final_df['Neighbourhood']=='Not assigned', final_df['Borough'], final_df['Neighbourhood'])
final_df.head(12)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Final table shape:

In [15]:
final_df.shape

(103, 3)

<hr>

<center><h1> This is Section 2 for the Segmenting and Clustering Neighbourhoods in Toronto Assignment</h1>

<hr>

Creating csv file:

In [16]:
lat_long=pd.read_csv('https://cocl.us/Geospatial_data')

Displaying table:

In [17]:
lat_long.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging the tables:

In [18]:
merged_df=pd.merge(final_df, lat_long)

Display new merged table:

In [19]:
merged_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


<hr>

<center><h1> This is Section 3 for the Segmenting and Clustering Neighbourhoods in Toronto Assignment</h1>

<hr>

Selecting and displaying all Postal Codes with Toronto stated in their Boroughs:

In [20]:
tor_df=merged_df[merged_df['Borough'].str.contains('Toronto')]
tor_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


Inspecting the size of the tor_df table:

In [21]:
tor_df.shape

(39, 5)

Retrieving the mean of the Lattitude and Longitude:

In [22]:
tor_df.describe()

,Latitude,Longitude
count,39.000000,39.000000
mean,43.667135,-79.389873
std,0.023478,0.037451
min,43.628947,-79.484450
25%,43.649765,-79.405678
50%,43.662301,-79.387383
75%,43.677957,-79.376474
max,43.728020,-79.293031


Folium Map display of the Greater Toronto Area (GTA):

In [23]:
latitude=43.667135
longitude=-79.389873

greater_toronto_area = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(tor_df['Latitude'], tor_df['Longitude'], tor_df['Borough'], tor_df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(greater_toronto_area)  
    
greater_toronto_area

Run k-means to cluster the neighborhood into 5 clusters:

In [24]:
kclusters = 5

toronto_grouped_clustering = tor_df.drop(['Postal Code','Borough', 'Neighbourhood'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 4, 0, 0, 3, 0, 1])

Create a new dataframe that includes the cluster:

In [25]:
tor_df.insert(0, 'Cluster Labels', kmeans.labels_)

Display the new dataframe:

In [26]:
tor_df.head()

,Cluster Labels,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,0,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,0,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,0,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,4,M4E,East Toronto,The Beaches,43.676357,-79.293031


Create the map cluster displaying the neighbourhoods in Toronto City:

In [27]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_df['Latitude'], tor_df['Longitude'], tor_df['Neighbourhood'], tor_df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters